# Resumir varios documentos

## Instalación de paquetes
Si estás corriendo este notebook en Google Colab, corre la siguiente celda para instalar los paquetes necesarios.

In [1]:
# %pip install langchain langchain_community langchain_openai

In [1]:
# Corre esta celda solo si tienes un archivo .env configurado
from dotenv import load_dotenv
load_dotenv()

True

Puedes resumir fácilmente información de varias fuentes de datos, como Wikipedia o archivos locales en formatos de Microsoft Word, PDF y texto. Para generar un solo resumen de diversos documentos utilizaremos la técnica ***Refine***, que consiste en construir el resumen final refinándolo gradualmente con cada fragmento del documento, similar al enfoque map-reduce que vimos en el laboratorio pasado.
 <br /> <br /> <br /> 
El proceso de Refine se lleva a cabo de la siguiente manera:<br /> <br /> 

1. Dividir los documentos en fragmentos pequeños (por ejemplo, de 3,000 tokens).<br /> <br /> 
2. Inicializar un resumen vacío o un borrador inicial del resumen final.<br /> <br /> 
3. Iterar sobre cada fragmento de documento:<br /> <br /> 
    1. Enviar el fragmento actual y el resumen existente al modelo de lenguaje (LLM).
    2. El LLM genera un nuevo resumen que integra y mejora el anterior, incorporando la información del fragmento actual.
    3. Actualizar el resumen final con este nuevo resumen refinado.<br /> <br /> 
4. Repetir el paso 3 hasta procesar todos los fragmentos.<br /> <br /> 
5. Obtener un resumen final que captura la información esencial de todos los documentos.

## Carga de documentos

In [3]:
# Instala los paquetes necesarios para los Loaders
# %pip install wikipedia docx2txt pypdf

In [26]:
from langchain.document_loaders import WikipediaLoader
 
wikipedia_loader = WikipediaLoader(query="Pueblo Añú", load_max_docs=2)
wikipedia_docs = wikipedia_loader.load()

In [27]:
from langchain.document_loaders import Docx2txtLoader
from langchain.document_loaders import PyPDFLoader
from langchain.document_loaders import TextLoader
from langchain_core.output_parsers import StrOutputParser
 
word_loader = Docx2txtLoader("../datasets/anu/pueblo_anu.docx")
word_docs = word_loader.load()
 
pdf_loader = PyPDFLoader("../datasets/anu/pautas-crianza-pueblo-anu-venezuela_reducido.pdf")
pdf_docs = pdf_loader.load()
 
txt_loader = TextLoader("../datasets/anu/pueblo_anu.txt")
txt_docs = txt_loader.load()

In [28]:
len(wikipedia_docs), len(word_docs), len(pdf_docs), len(txt_docs)

(2, 1, 5, 1)

El framework de LangChain ofrece numerosos Loaders (cargadores) para recuperar contenido de diversas fuentes de datos. Te animo a explorar la lista y a experimentar con otros Loaders:

https://python.langchain.com/docs/integrations/document_loaders

In [29]:
# Ahora con los documentos cargados, podemos consolidarla en una sola
all_docs = wikipedia_docs + word_docs + pdf_docs + txt_docs

## Implementación de Refine

In [30]:
import os
from langchain_openai import ChatOpenAI
from langchain_core.prompts import PromptTemplate

In [31]:
llm = ChatOpenAI(
    model=os.getenv("MODEL"),
    openai_api_key=os.getenv("LIA_API_KEY"),
    openai_api_base=os.getenv("LIA_API_BASE"),
    temperature=0.6,
)

In [32]:
# Ahora, define la cadena, con el prompt correspondiente, para resumir documentos individuales
plantilla_resumen_doc = """Escribe un resumen conciso del siguiente texto:
{texto}
RESUMEN DEL TEXTO:"""
prompt_resumen_doc = PromptTemplate.from_template(plantilla_resumen_doc)
 
cadena_resumen_doc = prompt_resumen_doc | llm

In [33]:
plantilla_resumen_refinado = """
Debes producir un resumen final a partir del resumen refinado actual 
que se ha generado hasta ahora y a partir del contenido de un documento adicional.
Este es el resumen refinado actual generado hasta ahora: {resumen_refinado_actual}
Este es el contenido del documento adicional: {texto}
Utiliza solo el contenido del documento adicional si es útil, 
de lo contrario, devuelve el resumen completo actual tal como está."""
 
prompt_resumen_refinado = PromptTemplate.from_template(plantilla_resumen_refinado)
 
cadena_refinado = prompt_resumen_refinado | llm | StrOutputParser()

Finalmente, define una función que recorra cada documento, lo resuma usando el `doc_summary_chain`, y refine el resumen general utilizando el `refine_chain`:

In [34]:
def refine_summary(docs):
 
    intermediate_steps = []
    current_refined_summary = ''
    for doc in docs:
        intermediate_step = \
           {"resumen_refinado_actual": current_refined_summary, 
            "texto": doc.page_content}
        intermediate_steps.append(intermediate_step)
        
        current_refined_summary = cadena_refinado.invoke(intermediate_step)
        
    return {"resumen_final": current_refined_summary,
            "pasos_intermedios": intermediate_steps}

In [35]:
# Ahora, ejecuta la función para obtener el resumen final
resumen_final = refine_summary(all_docs)

In [36]:
resumen_final["resumen_final"]

'El resumen final, integrando el contenido del documento adicional sobre el pueblo Añú, es el siguiente:\n\nLa clasificación de grupos étnicos ha sido objeto de constante debate, donde la pertenencia a un grupo étnico se asocia con la ascendencia compartida, historia, territorio, lengua o dialecto, y patrimonio cultural. Este concepto de cultura abarca aspectos como religión, mitología, rituales, gastronomía y estilo de vestimenta. Los grupos étnicos pueden dividirse en subgrupos y su identificación como grupos independientes puede variar según la fuente consultada.\n\nUn ejemplo de grupo étnico es el pueblo Añú, también conocido como Paraujano, una de las cinco comunidades indígenas de Venezuela, específicamente del Estado Zulia. Con una población total que se estima en al menos 15,000 personas, los Añú hablan el idioma añú, perteneciente a la familia arawak. Su designación significa "Gente de agua" o "Gente del mar", y se les conoce también como Paraujanos, término que deriva de su r